In [20]:
!pip install psycopg2-binary sqlalchemy --quiet

In [29]:
import os
import pandas as pd
from sqlalchemy import create_engine
import socket

# Force IPv4 to avoid Colab IPv6 issues
orig_getaddrinfo = socket.getaddrinfo
def ipv4_getaddrinfo(*args, **kwargs):
    return [ai for ai in orig_getaddrinfo(*args, **kwargs) if ai[0] == socket.AF_INET]
socket.getaddrinfo = ipv4_getaddrinfo

# ✅ Use your full PostgreSQL connection URL
db_url = os.DBURL;
# Create engine and connect
try:
    engine = create_engine(db_url)
    with engine.connect() as connection:
        print("✅ Connection successful!")

        # Run a query
        df = pd.read_sql("SELECT * FROM jobs LIMIT 10;", connection)
        display(df)

except Exception as e:
    print(f"❌ Connection failed: {e}")


✅ Connection successful!


,title,company,description,url,date,location,Excel,SQL,Python,Tableau,Power BI,Pandas,R,Git,Azure,AWS,GCP,AI,skill_count
0,Lead Data Engineer,Open Architects,Open Architects (OA) seeks a Lead Data Enginee...,https://remoteOK.com/remote-jobs/remote-lead-d...,2025-06-02 22:32:54+00:00,,False,False,True,False,False,False,False,True,False,False,False,False,2
1,Data Analyst Canada Wide,Newton,Say hello to Newton! We're changing how Canadi...,https://remoteOK.com/remote-jobs/remote-data-a...,2025-06-02 13:00:11+00:00,"Toronto, Ontario",False,True,True,False,False,False,False,False,False,False,False,False,2
2,Data Scientist,The Voleon Group,Voleon is a technology company that applies st...,https://remoteOK.com/remote-jobs/remote-data-s...,2025-06-01 05:00:02+00:00,United States or Remote,False,True,False,False,False,True,False,True,False,False,False,True,4
3,Data Analyst,Foodsmart,About us:Foodsmart is the leading telenutritio...,https://remoteOK.com/remote-jobs/remote-data-a...,2025-05-21 00:00:09+00:00,US,False,True,True,True,True,False,True,False,False,False,False,False,5
4,Game Data Analyst,athinkingape,"At A Thinking Ape, data is a core part of how ...",https://remotive.com/remote-jobs/data/game-dat...,2025-06-09 12:50:56+00:00,"USA, Canada",False,True,True,True,True,False,False,False,False,False,False,False,4
5,Senior Data Scientist,BaxEnergy,We’re looking for a senior Data Scientist to j...,https://remotive.com/remote-jobs/software-dev/...,2025-06-09 10:29:12+00:00,European timezones,False,False,True,False,False,True,False,False,False,False,False,False,2
6,Senior Data Engineer,DICK'S Sporting Goods,This description is a summary of our understa...,https://remotive.com/remote-jobs/software-dev/...,2025-06-09 08:50:25+00:00,USA,False,True,True,False,False,False,False,False,False,False,False,False,2
7,Data Analyst,Valsoft Corporation Inc.,We are looking for a Data Analyst to join our ...,https://remotive.com/remote-jobs/data/data-ana...,2025-06-08 10:50:55+00:00,Spain,True,True,False,False,False,False,False,False,False,False,False,True,3
8,Member Operations Analyst,Garner Health,Healthcare quality is declining and soaring co...,https://remotive.com/remote-jobs/customer-supp...,2025-06-08 10:50:21+00:00,USA,False,False,False,False,False,False,False,False,False,False,False,False,0
9,Principal Data Engineer,The Nuclear Company,The Nuclear Company is the fastest growing sta...,https://remotive.com/remote-jobs/software-dev/...,2025-06-07 20:50:40+00:00,USA,False,True,True,False,False,False,False,False,True,True,True,False,5


Top 10 job titles

In [41]:
sql_query_title = "SELECT title, COUNT(*) AS cnt FROM jobs GROUP BY title ORDER BY cnt DESC LIMIT 10;"
df_titles = pd.read_sql(sql_query_title, engine)
display(df_titles)

,title,cnt
0,Senior Data Engineer,6
1,Data Scientist,6
2,Senior Data Scientist,3
3,Data Analyst,3
4,Senior Data Analyst,3
5,Sr. Data Scientist,2
6,"Staff, Data Scientist",2
7,Lead Data Engineer,2
8,Data Engineer,2
9,Customer Data Engineer,1


Top 10 Companies Hiring

In [43]:
sql_query_company = """
SELECT company, COUNT(*) AS count
FROM jobs
GROUP BY company
ORDER BY count DESC
LIMIT 10;
"""
df_company = pd.read_sql(sql_query_company, engine)
display(df_company)

,company,count
0,Walmart,4
1,Mercury,2
2,,2
3,Verizon,1
4,tires-easy.com,1
5,Foodsmart,1
6,Welcome,1
7,Newton,1
8,Valsoft Corporation Inc.,1
9,Datenschmiede.ai GmbH,1


Most in-demand skills

In [44]:
query = """
SELECT
    SUM(CASE WHEN "Python" THEN 1 ELSE 0 END) AS python,
    SUM(CASE WHEN "SQL" THEN 1 ELSE 0 END) AS sql,
    SUM(CASE WHEN "Excel" THEN 1 ELSE 0 END) AS excel,
    SUM(CASE WHEN "Tableau" THEN 1 ELSE 0 END) AS tableau,
    SUM(CASE WHEN "Power BI" THEN 1 ELSE 0 END) AS power_bi,
    SUM(CASE WHEN "Pandas" THEN 1 ELSE 0 END) AS pandas,
    SUM(CASE WHEN "R" THEN 1 ELSE 0 END) AS r,
    SUM(CASE WHEN "Git" THEN 1 ELSE 0 END) AS git,
    SUM(CASE WHEN "Azure" THEN 1 ELSE 0 END) AS azure,
    SUM(CASE WHEN "AWS" THEN 1 ELSE 0 END) AS aws,
    SUM(CASE WHEN "GCP" THEN 1 ELSE 0 END) AS gcp,
    SUM(CASE WHEN "AI" THEN 1 ELSE 0 END) AS ai
FROM jobs;
"""

df_skills = pd.read_sql(query, engine)
display(df_skills)

,python,sql,excel,tableau,power_bi,pandas,r,git,azure,aws,gcp,ai
0,39,37,6,15,9,7,11,5,10,16,5,18


Average Skill Count per Job

In [46]:
query = """
SELECT ROUND(AVG(skill_count), 2) AS avg_skill_count
FROM jobs;
"""
df_avg = pd.read_sql(query, engine)
print(df_avg)


   avg_skill_count
0             2.87


In [52]:
# engine.close()
# conn.close()